In [56]:
import functools
from collections import defaultdict
import numpy as np

import tensorflow as tf

from evaluation import precision_recall_f1

print(tf.__version__)

1.15.0


In [0]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            # Replace all urls with <URL> token
            # Replace all users with <USR> token

            ######################################
            ######### YOUR CODE HERE #############
            ######################################
            if token.startswith('@'):
                token = '<USR>'
            if token.startswith('http://') or token.startswith('https://'):
                token = '<URL>'
            
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

In [0]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

In [10]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



In [0]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    tokens_or_tags = [*functools.reduce(lambda x, y: x + y, tokens_or_tags)]
    unique_tokens_or_tags = [x for x in set(tokens_or_tags)
                                if x not in special_tokens]
    for i, tok in enumerate(special_tokens + unique_tokens_or_tags):
        tok2idx[tok] = i
        idx2tok.append(tok)
    
    return tok2idx, idx2tok

In [0]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens,
                                  special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

In [0]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

In [0]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """Generates padded batches of tokens and tags."""
    
    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        # Fill in the data into numpy nd-arrays filled with padding indices.
        x = np.ones([current_batch_size, max_len_token],
                    dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token],
                    dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths

In [0]:
class BiLSTMModel():

    def __init__(self, vocabulary_size, n_tags, embedding_dim,
                 n_hidden_rnn, PAD_index):
        self.__declare_placeholders()
        self.__build_layers(vocabulary_size, embedding_dim,
                            n_hidden_rnn, n_tags)
        self.__compute_predictions()
        self.__compute_loss(n_tags, PAD_index)
        self.__perform_optimization()
    
    def __declare_placeholders(self):
        # Placeholders for input and ground truth output.
        self.input_batch = tf.placeholder(dtype=tf.int32,
                                          shape=[None, None],
                                          name='input_batch') 
        self.ground_truth_tags = tf.placeholder(dtype=tf.int32,
                                                shape=[None, None],
                                                name='ground_truth_tags')
    
        # Placeholder for lengths of the sequences.
        self.lengths = tf.placeholder(dtype=tf.int32,
                                      shape=[None],
                                      name='lengths') 
        
        # Placeholder for a dropout keep probability. If we don't feed
        # a value for this placeholder, it will be equal to 1.0.
        self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32),
                                                      shape=[])
        
        # Placeholder for a learning rate.
        self.learning_rate_ph = tf.placeholder(dtype=tf.float32,
                                               shape=[], 
                                               name='learning_rate_ph')
        
    def __build_layers(self, vocabulary_size, embedding_dim,
                     n_hidden_rnn, n_tags):
        # Create embedding variable
        initial_embedding_matrix = np.random.randn(vocabulary_size,
                                    embedding_dim) / np.sqrt(embedding_dim)
        embedding_matrix_variable = tf.Variable(initial_embedding_matrix,
                                                dtype=tf.float32,
                                                name='embeddings_matrix')
        # Create RNN cells
        forward_cell = tf.nn.rnn_cell.DropoutWrapper(
                            tf.nn.rnn_cell.LSTMCell(n_hidden_rnn),
                            input_keep_prob=self.dropout_ph,
                            output_keep_prob=self.dropout_ph,
                            state_keep_prob=self.dropout_ph)
        backward_cell = tf.nn.rnn_cell.DropoutWrapper(
                            tf.nn.rnn_cell.LSTMCell(n_hidden_rnn),
                            input_keep_prob=self.dropout_ph,
                            output_keep_prob=self.dropout_ph,
                            state_keep_prob=self.dropout_ph)
        
        # Shape: [batch_size, sequence_len, embedding_dim].
        embeddings =  tf.nn.embedding_lookup(embedding_matrix_variable,
                                             self.input_batch)
        
        # Shape: [batch_size, sequence_len, 2 * n_hidden_rnn].
        (rnn_output_fw, rnn_output_bw), _ = tf.nn.bidirectional_dynamic_rnn(
                                        forward_cell,
                                        backward_cell,
                                        embeddings,
                                        sequence_length=self.lengths,
                                        dtype=tf.float32)
        rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)

        self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)

    def __compute_predictions(self):
        softmax_output = tf.nn.softmax(self.logits)
        self.predictions = tf.math.argmax(softmax_output, axis=-1)

    def __compute_loss(self, n_tags, PAD_index):
        ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
        loss_tensor = tf.nn.softmax_cross_entropy_with_logits_v2(
            ground_truth_tags_one_hot, self.logits)
        mask = tf.cast(tf.not_equal(self.input_batch, PAD_index), tf.float32)
        self.loss = tf.reduce_mean(loss_tensor * mask)

    def __perform_optimization(self):
        self.optimizer =  tf.train.AdamOptimizer()
        self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
        
        # Gradient clipping only for gradients because compute_gradients method
        # also returns variables.
        clip_norm = tf.cast(1.0, tf.float32)
        self.grads_and_vars = [(tf.clip_by_norm(grad, clip_norm), var) 
                                for grad, var in self.grads_and_vars]
        self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

    def train_on_batch(self, session, x_batch, y_batch, lengths,
                       learning_rate, dropout_keep_probability):
        feed_dict = {self.input_batch: x_batch,
                     self.ground_truth_tags: y_batch,
                     self.learning_rate_ph: learning_rate,
                     self.dropout_ph: dropout_keep_probability,
                     self.lengths: lengths}
        
        session.run(self.train_op, feed_dict=feed_dict)

    def predict_for_batch(self, session, x_batch, lengths):
        feed_dict = {self.input_batch: x_batch,
                     self.lengths: lengths}
        
        predictions = session.run(self.predictions, feed_dict=feed_dict)
        return predictions

In [0]:
def predict_tags(model, session, token_idxs_batch, lengths): 
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session,
                                                x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]),
                                                        len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0],
                                               tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True,
                                  short_report=short_report)
    return results

In [0]:
tf.reset_default_graph()

model = BiLSTMModel(vocabulary_size=len(token2idx), n_tags=len(tag2idx),
                    embedding_dim=200, n_hidden_rnn=200,
                    PAD_index=token2idx['<PAD>'])

batch_size = 32
n_epochs = 10
learning_rate = 0.05
learning_rate_decay = 0.9
dropout_keep_probability = 0.6

In [67]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start training... \n')
for epoch in range(n_epochs):
    print('=' * 20 + f' Epoch {epoch+1} ' + f'of {n_epochs} ' + '=' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags,
               short_report=True)
    
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(batch_size, 
                                                       train_tokens,
                                                       train_tags):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate,
                             dropout_keep_probability)
        
    # Decaying the learning rate
    learning_rate = learning_rate / learning_rate_decay
    
print('...training finished.')

Start training... 

==================== Epoch 1 of 10 ====================
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 79204 phrases; correct: 182.

precision:  0.23%; recall:  4.05%; F1:  0.43

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 9612 phrases; correct: 28.

precision:  0.29%; recall:  5.21%; F1:  0.55

==================== Epoch 2 of 10 ====================
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 16 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; F1:  0.00

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 2 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; F1:  0.00

==================== Epoch 3 of 10 ====================
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 3494 phrases; correct: 470.

precision:  13.45%; recall:  10.47%; F1:  11.78

Validation data evaluation:
processed 12836 tokens with 537 phrases; fou

In [68]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, sess, train_tokens, train_tags,
                           short_report=False)

print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, sess, validation_tokens,
                                validation_tags, short_report=False)

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, sess, test_tokens, test_tags,
                          short_report=False)

-------------------- Train set quality: --------------------
processed 105778 tokens with 4489 phrases; found: 4624 phrases; correct: 4094.

precision:  88.54%; recall:  91.20%; F1:  89.85

	     company: precision:   92.25%; recall:   96.27%; F1:   94.22; predicted:   671

	    facility: precision:   90.77%; recall:   93.95%; F1:   92.33; predicted:   325

	     geo-loc: precision:   94.47%; recall:   97.69%; F1:   96.05; predicted:  1030

	       movie: precision:   42.42%; recall:   41.18%; F1:   41.79; predicted:    66

	 musicartist: precision:   75.13%; recall:   62.50%; F1:   68.24; predicted:   193

	       other: precision:   90.36%; recall:   91.68%; F1:   91.02; predicted:   768

	      person: precision:   93.22%; recall:   96.28%; F1:   94.73; predicted:   915

	     product: precision:   71.83%; recall:   88.99%; F1:   79.49; predicted:   394

	  sportsteam: precision:   82.13%; recall:   88.94%; F1:   85.40; predicted:   235

	      tvshow: precision:   40.74%; recall:  